## Initalization

In [ ]:
# Standard includes
%matplotlib inline
# %matplotlib widget
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Style setup
import seaborn as sns

sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

def darken_color(color, p):
    return (color[0] * p, color[1] * p, color[2] * p)


import matplotlib as mpl

colors = sns.color_palette("muted") + [(0.1, 0.1, 0.1)]
for code, color in zip(["bd", "gd", "rd", "md", "yd", "cd", "kd"], colors):
    rgb = mpl.colors.colorConverter.to_rgb(darken_color(color, 0.8))
    mpl.colors.colorConverter.colors[code] = rgb
    mpl.colors.colorConverter.cache[code] = rgb

blue = (114 / 256, 147 / 256, 203 / 256)
orange = (225 / 256, 151 / 256, 76 / 256)
green = (132 / 256, 186 / 256, 91 / 256)
red = (211 / 256, 94 / 256, 96 / 256)
grey = (128 / 256, 133 / 256, 133 / 256)
violet = (144 / 256, 103 / 256, 167 / 256)
brown = (171 / 256, 104 / 256, 87 / 256)
yellow = (204 / 256, 194 / 256, 16 / 256)

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams["figure.figsize"] = [6, 6]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["savefig.format"] = "png"

PLOT_DIR = Path("plots")
PLOT_DIR.mkdir(parents=True, exist_ok=True)

# Plots

### Data

In [ ]:
df_base = pd.read_csv(Path("..") / "data" / "DATA_baseline.CSV", header=None, index_col=0)
df_base2 = pd.read_csv(Path("..") / "data" / "DATA_baseline2.CSV", header=None, index_col=0)
df_eve = pd.read_csv(Path("..") / "data" / "DATA_evening.CSV", header=None, index_col=0)
df_last = pd.read_csv(Path("..") / "data" / "DATA_last_stand.CSV", header=None, index_col=0)
df_mike = pd.read_csv(Path("..") / "data" / "DATA_mike.CSV", header=None, index_col=0)
df_noferro = pd.read_csv(Path("..") / "data" / "DATA_noferro.CSV", header=None, index_col=0)
df_nomag = pd.read_csv(Path("..") / "data" / "DATA_nomagnets.CSV", header=None, index_col=0)
df_space = pd.read_csv(Path("..") / "data" / "DATA_space.CSV", header=None, index_col=0)

### Setup

In [ ]:
interval = 0.5  # [ms]
delay = 1.2  # [ms]
times = np.array(range(29)) * interval + delay

In [ ]:
def overlay_plot(data, savefig=None):
    i = 0
    while i < 40 and i < len(data):
        plt.plot(times, data.iloc[i])
        i+=1

    plt.xlabel("Time [ms]")
    plt.ylabel("Voltage [V]")

    if savefig:
        plt.savefig(PLOT_DIR / savefig)

In [ ]:
def overlay_plot_raw(data, savefig=None):
    i = 0
    while i < 40 and i < len(data):
        data.iloc[i].plot(xlabel="Measurement", ylabel="Voltage [V]")
        i+=1
        
    if savefig:
        plt.savefig(PLOT_DIR / savefig)

In [ ]:
def plot_averaged(data, savefig=None):
    means = data.mean()
    sigmas = data.std()
    
    plt.plot(times, means)
    plt.fill_between(times, means - sigmas, means + sigmas, alpha=0.5)
    
    plt.xlabel("Time [ms]")
    plt.ylabel("Voltage [V]")

    if savefig:
        plt.savefig(PLOT_DIR / savefig)

In [ ]:
def plot_averaged_smooth(data, label=None, color=None, savefig=None):
    means = data.mean()
    sigmas = data.std()
    
    from scipy.interpolate import make_interp_spline, BSpline

    times_smooth = np.linspace(times[0], times[-1], 300) 

    spline_means = make_interp_spline(times, means, k=3)  # type: BSpline
    means_smooth = spline_means(times_smooth)
    
    spline_upper = make_interp_spline(times, means + sigmas, k=3)  # type: BSpline
    upper_smooth = spline_upper(times_smooth)
    
    spline_lower = make_interp_spline(times, means - sigmas, k=3)  # type: BSpline
    lower_smooth = spline_lower(times_smooth)

    plt.plot(times_smooth, means_smooth, label=label, color=color)
    plt.fill_between(times_smooth, lower_smooth, upper_smooth, alpha=0.3, color=color)
    
    plt.xlabel("Time [ms]")
    plt.ylabel("Voltage [V]")

    if savefig:
        plt.savefig(PLOT_DIR / savefig)

### Raw

In [ ]:
overlay_plot(df_last.iloc[:-2:2], "last")

In [ ]:
overlay_plot(df_last.iloc[1:-2:2])

In [ ]:
overlay_plot_raw(df_mike.query("@df_mike[1]==1").drop(columns=[1]), "mike_coil1")

In [ ]:
overlay_plot_raw(df_mike.query("@df_mike[1]==2").drop(columns=[1]), "mike_coil2")

In [ ]:
overlay_plot(df_eve.iloc[:-2:2])

In [ ]:
overlay_plot(df_eve.iloc[1:-2:2])

In [ ]:
overlay_plot(df_base.iloc[:-2:2], "base_even")

In [ ]:
overlay_plot(df_base.iloc[1:-2:2], "base_odd")

In [ ]:
overlay_plot(df_noferro.iloc[:-2:2], "noferro_even")

In [ ]:
overlay_plot(df_noferro.iloc[1:-2:2], "noferro_odd")

In [ ]:
overlay_plot(df_nomag.iloc[:-2:2], "nomag_even")

In [ ]:
overlay_plot(df_nomag.iloc[1:-2:2], "nomag_odd")

In [ ]:
overlay_plot(df_base2.iloc[:-2:2], "base2_even")

In [ ]:
overlay_plot(df_space.iloc[1:-2:2], "space_odd")

In [ ]:
overlay_plot(df_space.iloc[:-2:2], "space_even")

In [ ]:
overlay_plot(df_space.iloc[1:-2:2], "space_odd")

### Averaged

In [ ]:
plot_averaged_smooth(df_base.iloc[:-2:2], "baseline", color=colors[0])
plot_averaged_smooth(df_noferro.iloc[:-2:2], "no ferrite", color=colors[1])
plot_averaged_smooth(df_nomag.iloc[:-2:2], "no solenoid", color=colors[2])
plt.legend()
plt.savefig(PLOT_DIR / "comparison_virginia_even_smooth")

In [ ]:
plot_averaged_smooth(df_base.iloc[1:-2:2], "baseline", color=colors[0])
plot_averaged_smooth(df_noferro.iloc[1:-2:2], "no ferrite", color=colors[1])
plot_averaged_smooth(df_nomag.iloc[1:-2:2], "no solenoid", color=colors[2])
plt.legend()
plt.savefig(PLOT_DIR / "comparison_virginia_odd_smooth")

In [ ]:
plot_averaged_smooth(df_space.iloc[:-2:2], "space", color=colors[0])
plt.savefig(PLOT_DIR / "space_even_smooth")

In [ ]:
plot_averaged_smooth(df_space.iloc[1:-2:2], "space", color=colors[0])
plt.savefig(PLOT_DIR / "space_odd_smooth")

In [ ]:
plot_averaged_smooth(df_base2.iloc[:-2:2], "baseline", color=colors[1])
plot_averaged_smooth(df_space.iloc[:-2:2], "space", color=colors[0])
plt.legend()
plt.savefig(PLOT_DIR / "comparison_even_smooth")

In [ ]:
plot_averaged_smooth(df_base2.iloc[1:-2:2], label="baseline", color=colors[1])
plot_averaged_smooth(df_space.iloc[1:-2:2], label="space", color=colors[0])
plt.legend(loc="lower right")
plt.savefig(PLOT_DIR / "comparison_odd_smooth")